# Very First Look at very first ComCam Data with mosaic with DM code

- Goal verification of early data : Early nights
- Restart the Kernel first
- Applyied to postISRCCD only (do CALEXP in other nb)

- LSST Doc for Mosaic : https://pipelines.lsst.io/v/d_2024_06_11/py-api/lsst.pipe.tasks.visualizeVisit.VisualizeMosaicExpTask.html

- author : Sylvie Dagoret-campagne
- affiliation : IJCLab/in2p3/CNRS
- creation date : 2024-10-30
- update : 2024-10-30 : Use DM Mosaic building for showing the focal plane.
- last update : 2025-01-09 : fix the date decoding ix the date decoding (w_2024_50)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.dates as mdates
import matplotlib.ticker
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm
from matplotlib.colors import ListedColormap
from matplotlib import colors

%matplotlib inline

In [ ]:
plt.rcParams["figure.figsize"] = (18,8)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'
plt.rcParams['legend.fontsize']=  12
plt.rcParams['font.size'] = 12

In [ ]:
import numpy as np
import pandas as pd
from astropy.time import Time

In [ ]:
from lsst.daf.butler import Butler 

In [ ]:
# LSST Display
# The advantage to use firefly is that the firefly display can handle a lotsof images
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [ ]:
from lsst.pipe.tasks.visualizeVisit import (
    VisualizeBinExpConfig,
    VisualizeBinExpTask,
    VisualizeMosaicExpConfig,
    VisualizeMosaicExpTask,
)

## Configuration

- Use the tag report to guess the collections in which we exect data :  https://usdf-rsp-dev.slac.stanford.edu/times-square/github/lsst-dm/vv-team-notebooks/TargetReport

In the case of LSSTComCam:

- https://usdf-rsp-dev.slac.stanford.edu/times-square/github/lsst-dm/vv-team-notebooks/TargetReport?day_obs=2024-10-24&instrument=LSSTComCam&repo=embargo_new&collection=LSSTComCam%2Fprompt%2Foutput-2024-10-24&collection_sky=LSSTComCamSim%2Fruns%2FDRP%2FOR4%2Fw_2024_25%2FDM-45066&skymap_name=ops_rehersal_prep_2k_v1&col_sciprog=science_program&col_target=target&col_filter=filter&col_id=id&ts_hide_code=1

### Info from slack channel  #validation_team
day_obs:  2024-10-24  
instrument:  LSSTComCam  
repo:  embargo_new  
collection:  LSSTComCam/prompt/output-2024-10-24  
collection_sky:  LSSTComCamSim/runs/DRP/OR4/w_2024_25/DM-45066  
skymap_name:  ops_rehersal_prep_2k_v1  
col_sciprog:  science_program  
col_target:  target  
col_filter:  filter  
col_id:  id

In [ ]:
repo = 'embargo_new'
instrument = "LSSTComCam"
collection = 'LSSTComCam/prompt/output-2024-10-24'

butler = Butler(repo)

## Query about the collections available

- select collections with LSSTComCam

In [ ]:
for _ in butler.registry.queryCollections():
    if 'LSSTComCam/' in _:
        print(_)

- From the list available today , we probably have to look inside the colleciton `LSSTComCam/nightlyValidation`

## Select the Camera

In [ ]:
# mandatory to get the focal plane setup
camera = butler.get("camera", collections="LSSTComCam/defaults", instrument="LSSTComCam")

In [ ]:
camera

## Select the collection

In [ ]:
#the_collection = "LSSTComCam/runs/nightlyValidation_20241024_20241025/w_2024_42/DM-47059-test" 
#the_collection =   "LSSTComCam/prompt/output-2024-10-24"
the_collection = "LSSTComCam/nightlyValidation"

## Create the butler on the selected collection

In [ ]:
butler = Butler(repo, collections=the_collection)
registry = butler.registry

In [ ]:
for datasetType in registry.queryDatasetTypes():
    if registry.queryDatasets(datasetType, collections= the_collection).any(execute=False, exact=False):
        # Limit search results to the data products
        if ('_config' not in datasetType.name) and ('_log' not in datasetType.name) and ('_metadata' not in datasetType.name) and ('_resource_usage' not in datasetType.name):
            print(datasetType)

- Notice no visit-Table is available. Thus to know which visiId are existing, we will use the registry later

## Extract the list of visitId from the butler's registry 

### Get the list of information that can be extracted from the registryfor each exposure

In [ ]:
print(registry.dimensions["exposure"].RecordClass.fields)

### Create the pandas dataframe from the information inside the butler's registry

In [ ]:
df_exposure = pd.DataFrame(columns=['id', 'obs_id','day_obs', 'seq_num','time_start','time_end' ,'type', 'target','filter','zenith_angle','expos','ra','dec','skyangle','azimuth','zenith','science_program','jd','mjd'])

### Fill the pandas dataframe from the records inside the butler's registry for each exposure

In [ ]:
for count, info in enumerate(registry.queryDimensionRecords('exposure',where = where_expr)):  
    try:
        id_ = info.id
        obs_id_ = info.obs_id
        day_obs_ = info.day_obs
        seq_num_ = info.seq_num
        timespan_ = info.timespan

        timespan_begin_ = pd.to_datetime(info.timespan.begin.to_datetime())
        timespan_end_ = pd.to_datetime(info.timespan.end.to_datetime())
        timespan_jd_ = timespan_.begin.jd
        timespan_mjd_ = timespan_.begin.mjd
        
        
        df_exposure.loc[count] = [info.id, info.obs_id, info.day_obs, info.seq_num,timespan_begin_,timespan_end_ ,info.observation_type, info.target_name, info.physical_filter, info.zenith_angle, \
                             info.exposure_time,info.tracking_ra, info.tracking_dec, info.sky_angle,info.azimuth ,info.zenith_angle, info.science_program,
                             timespan_jd_,timespan_mjd_  ]
    except:
        print(">>>   Unexpected error:", sys.exc_info()[0])
        info_timespan_begin_to_string = "2021-01-01 00:00:00.00"
        info_timespan_end_to_string = "2051-01-01 00:00:00.00"
        info_timespan_begin_jd = 0
        info_timespan_begin_mjd = 0
        df_exposure.loc[count] = [info.id, info.obs_id, info.day_obs, info.seq_num,
                                  pd.to_datetime(info_timespan_begin_to_string),
                                  pd.to_datetime(info_timespan_end_to_string) ,
                                  info.observation_type, info.target_name, 
                                  info.physical_filter, info.zenith_angle, \
                             info.exposure_time,info.tracking_ra, info.tracking_dec, info.sky_angle,info.azimuth ,info.zenith_angle, info.science_program,
                             info_timespan_begin_jd,info_timespan_begin_mjd ]
 

    if count < 2:
        print("-----------------------------------------------------",count,"---------------------------------------------------------")
        print(info)
        print("\t id:                  ",info.id)
        print("\t day_obs:             ",info.day_obs)
        print("\t seq_num:             ",info.seq_num)
        print("\t type-of-observation: ",info.observation_type)
        print("\t target:              ",info.target_name)

        timespan_ = info.timespan
        timespan_begin_ = pd.to_datetime(info.timespan.begin.to_datetime())
        timespan_end_ = pd.to_datetime(info.timespan.end.to_datetime())
        timespan_jd_ = timespan_.begin.jd
        timespan_mjd_ = timespan_.begin.mjd
        
        mjd = timespan_mjd_ 
        jd = timespan_jd_ 
        print("MJD,JD : ",mjd,jd)
        
 

#### convert some columns into integer

In [ ]:
df_exposure = df_exposure.astype({"id": int,'day_obs': int,'seq_num':int})

In [ ]:
where_expr = "instrument='LSSTComCam'"

### Find the different type of exposures

In [ ]:
df_exposure["type"].unique()

### Select the science exposure

In [ ]:
df_science = df_exposure[df_exposure.type == 'science']
df_science.reset_index(drop=True, inplace=True)

In [ ]:
df_science

## Example for viewing one image

### Select a date and a visitId

In [ ]:
visitId = 2024102800113 	


### Extract the exposures postiSRCCD from the selected visitID

In [ ]:
def retrieve_images(visitId,instrument=instrument, butler= butler,collection=the_collection,datatype='postISRCCD'):
    the_date = visitId//100000 
    where_clause = f"instrument=\'LSSTComCam\' AND exposure.day_obs={the_date}"
    dataId = {'visit': visitId, 'instrument':instrument}
    
    datasetRefs = butler.registry.queryDatasets(datatype, dataId=dataId, collections  = collection)
    
    
    for i, ref in enumerate(datasetRefs):
        print(ref.dataId)
        exposure = ref.dataId["exposure"]
        detector = ref.dataId["detector"]
        physical_filter = ref.dataId["physical_filter"]
        band = ref.dataId["band"]
        #the_image  = butler.get(ref)
        #image_dict[detector]  =  the_image

    return exposure,band,physical_filter,datasetRefs


In [ ]:
exposure,band,physical_filter,datasetRefs = retrieve_images(visitId)

## View all the images of the focal surface for that vist

### Plot

In [ ]:
the_title = f"postisrccd :: id = {visitId}, band = {band}"

- from code snippet on slack : https://rubin-obs.slack.com/archives/C07TD4VUJ3X/p1729878639075269
  
      from lsst.daf.butler import Butler
      from lsst.pipe.tasks.visualizeVisit import (
      VisualizeBinExpConfig,
      VisualizeBinExpTask,
      VisualizeMosaicExpConfig,
      VisualizeMosaicExpTask,
      )

      butler = Butler("/repo/embargo_new")

      dataset_refs = butler.query_datasets("postISRCCD", collections="u/mrawls/day1-streaks",     where="instrument='LSSTComCam' AND exposure=2024102400128")
      exposures = [butler.get(dataset_ref) for dataset_ref in dataset_refs]
      camera = butler.get("camera", collections="LSSTComCam/defaults", instrument="LSSTComCam")

      binning = 4  # default is 8 pixels

      visualizeBinExpConfig = VisualizeBinExpConfig()
      visualizeBinExpConfig.binning = binning
      visualizeBinExpTask = VisualizeBinExpTask(config=visualizeBinExpConfig)
      exposures_binned = [visualizeBinExpTask.run(inputExp = exposure, camera=camera).outputExp for exposure in exposures]

      visualizeMosaicExpConfig = VisualizeMosaicExpConfig()
      visualizeMosaicExpConfig.binning = binning
      visualizeMosaicExpTask = VisualizeMosaicExpTask(config=visualizeMosaicExpConfig)
       mosaic = visualizeMosaicExpTask.run(inputExps=exposures_binned, camera=camera).outputData

In [ ]:
def MakeMosaic(datasetRefs,camera, binning = 8):
    """
    binning default : 8 
    """

    exposures = [butler.get(dataset_ref) for dataset_ref in datasetRefs]
    visualizeBinExpConfig = VisualizeBinExpConfig()
    visualizeBinExpConfig.binning = binning
    visualizeBinExpTask = VisualizeBinExpTask(config=visualizeBinExpConfig)
    exposures_binned = [visualizeBinExpTask.run(inputExp = exposure, camera=camera).outputExp for exposure in exposures]

    visualizeMosaicExpConfig = VisualizeMosaicExpConfig()
    visualizeMosaicExpConfig.binning = binning
    visualizeMosaicExpTask = VisualizeMosaicExpTask(config=visualizeMosaicExpConfig)
    the_mosaic = visualizeMosaicExpTask.run(inputExps=exposures_binned, camera=camera).outputData
    return the_mosaic


In [ ]:
mosaic = MakeMosaic(datasetRefs,camera, binning = 8)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,18))
display = afwDisplay.Display(frame=fig)
the_image = mosaic
plt.sca(ax)
display.scale('asinh', 'zscale')
display.mtv(the_image)
display.show_colorbar(False)
plt.gca().axis('off')
plt.suptitle(the_title)
plt.tight_layout()  
plt.show()

## Loop on visits

#### Select the visits

In [ ]:
df_sel = df_science[df_science["day_obs"] == 20241028]
list_of_visitId = list(df_sel.id)

In [ ]:
# loop on visit id
for idcount, visitId in enumerate(list_of_visitId):

    exposure,band,physical_filter,datasetRefs = retrieve_images(visitId)
    the_title = f"postisrccd :: id = {visitId}, band = {band}"
    mosaic = MakeMosaic(datasetRefs,camera, binning = 8)

    fig,ax = plt.subplots(1,1,figsize=(18,18))
    display = afwDisplay.Display(frame=fig)
    the_image = mosaic
    plt.sca(ax)
    display.scale('asinh', 'zscale')
    display.mtv(the_image)
    display.show_colorbar(False)
    plt.gca().axis('off')
    plt.suptitle(the_title)
    plt.tight_layout()  
    plt.show()